In [1]:
import pandas as pd
import torch as tc
import numpy as np
import multiprocessing
import re
from transformers import DistilBertTokenizerFast
import torch
from transformers import DistilBertForSequenceClassification
from transformers import Trainer, TrainingArguments

In [2]:
benign_df = pd.read_csv('../data/normal_df.csv').sample(frac=1).reset_index(drop=True)[:5000]
malicious_df = pd.read_csv('../data/malicious.csv')
sc_df = pd.concat([benign_df, malicious_df]).reset_index(drop=True)

In [3]:
sc_df.head()

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,contract_address,contract_name,contract_etherscan_label,contract_tag,contract_creator,contract_creation_tx,creation_bytecode,decompiled_opcodes,malicious,contract_creator_tag,source,notes,contract_creator_etherscan_label,__index_level_0__,_lua_timestamp,_lua_uuid,bytecode_from_web3
0,5710,5710,5710,0x5e539c591c3e26b3015905d049803e44f104bb9e,UniswapV2Pair,uniswap,Uniswap V2: F1 4,0xb73c668e268925d81d04574f2864a33677d9bded,0x910efb75ed733c597a3d4cf92fb67e30a27ab7ce646b...,0x60806040526001600c5534801561001557600080fd5b...,PUSH1 0x80 PUSH1 0x40 MSTORE PUSH1 0x01 PUSH1 ...,0,NaN,NaN,NaN,NaN,95663,2022-12-05T20:18:10,5e28926e-3d05-4a2b-83ff-b50b3f5b3b8b,0x608060405234801561001057600080fd5b5060043610...
1,29489,29489,29489,0x020171085bcd43b6fd36ad8c95ad61848b1211a2,yGift,yearn-finance,Yearn: yGifts,0x7a1057e6e9093da9c1d4c1d049609b6889fc4c67,0xb6630b9c7539ff1daddfe31659567b4bd70d44804700...,0x60806040523480156200001157600080fd5b50604080...,PUSH1 0x80 PUSH1 0x40 MSTORE CALLVALUE DUP1 IS...,0,NaN,NaN,NaN,NaN,1145,2022-12-05T20:18:10,a2e2ab0e-914d-4df4-8cc2-627ad2d19e5e,0x608060405234801561001057600080fd5b5060043610...
2,1243,1243,1243,0xd2a32c9a9f0eeb2514bce17a560b7d75457f6c8e,UniswapV2Pair,uniswap,Uniswap V2: ELE 6,0x7ace65840eb1819e3746fab72298312a585223aa,0x042e7952a97cf112e8daf38b107cdbe120be31aa45a5...,0x60806040526001600c5534801561001557600080fd5b...,PUSH1 0x80 PUSH1 0x40 MSTORE PUSH1 0x01 PUSH1 ...,0,NaN,NaN,NaN,NaN,17595,2022-12-05T20:18:10,9f6b1353-536a-44e8-81d6-d41bb4066e56,0x608060405234801561001057600080fd5b5060043610...
3,12791,12791,12791,0x4d30a2cee33634114deb0dd11ef2c59cde9d2ef6,GDLK,NaN,NaN,0xdbbc5151052c54acadbc2daebf97199842d80c6c,0x0a80d2cf7cf2f5e9b06ab0a6edc8ffdcc60e46f9f639...,0x600280546001600160a01b031990811661dead179091...,PUSH1 0x02 DUP1 SLOAD PUSH1 0x01 PUSH1 0x01 PU...,0,NaN,NaN,NaN,NaN,77423,2022-12-05T20:18:10,b5f35428-886e-470d-868a-cb75e0f97af0,0x6080604052600436106103dd5760003560e01c806377...
4,3123,3123,3123,0xb8cced52a76afadf607c595e9a12372fbf7f7f1a,UniswapV2Pair,uniswap,Uniswap V2: BIRDS 2,0xc13c75c8858a8d6e4262e4d614611d298ce48606,0x502d0719e9085d792e39b3e9d211f3f4d5a634f9a68f...,0x60806040526001600c5534801561001557600080fd5b...,PUSH1 0x80 PUSH1 0x40 MSTORE PUSH1 0x01 PUSH1 ...,0,NaN,NaN,NaN,NaN,56052,2022-12-05T20:18:10,9f2bcad5-5cb0-498b-8309-2dcd78fe36d0,0x608060405234801561001057600080fd5b5060043610...


In [4]:
sc_df = sc_df[['decompiled_opcodes', 'malicious']].sample(frac=1).reset_index(drop=True)

In [5]:
# sc_df = sc_df.drop(sc_df[sc_df.decompiled_code == "error"].index)
# sc_df = sc_df.drop(sc_df[sc_df.decompiled_code == "error ;"].index)
# sc_df = sc_df.reset_index()
# sc_df

In [6]:
tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")


In [7]:
inputs = tokenizer(sc_df.decompiled_opcodes.tolist(), padding="max_length", truncation=True)

In [8]:
class SmartContractsDataset(tc.utils.data.Dataset):
    def __init__(self, encodings, labels):
        # self.sc_df = sc_df
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        # code = self.sc_df.decompiled_code[idx]
        # code = simple_web3_code_decompile(code)
        # code = preprocess_code(code)
        # code = self.get_contract_as_image(code).transpose(2, 0, 1)
        # code = tc.tensor(code).type(tc.float32)
        # code = tokenizer(code).items()

        # mal = self.sc_df.malicious[idx]
        # mal = tc.tensor(mal).type(tc.float32)
        # return code, mal

        # item = {key: torch.tensor(val[idx]) for key, val in code}
        # item['label'] = mal
        # return item
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['label'] = torch.tensor(self.labels[idx])
        return item


In [9]:
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.weight', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier

In [10]:
device = "mps"


In [11]:
model.to(device)

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [12]:
scd = SmartContractsDataset(inputs, sc_df.malicious.tolist())
train_size = int(0.8 * len(scd))
val_size = int(0.1 * len(scd))
test_size = len(scd) - train_size - val_size
training_dataset, validation_dataset, testing_dataset = tc.utils.data.random_split(scd,
                                                                                   [train_size, val_size, test_size])


In [13]:
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=20,
    use_mps_device=True,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=training_dataset,
    eval_dataset=validation_dataset,
)

In [14]:
trainer.train()

/Users/venglov/miniconda3/envs/torch_transformers/lib/python3.10/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss


TrainOutput(global_step=5160, training_loss=0.046868195784613924, metrics={'train_runtime': 6623.257, 'train_samples_per_second': 12.438, 'train_steps_per_second': 0.779, 'total_flos': 1.091266430128128e+16, 'train_loss': 0.046868195784613924, 'epoch': 20.0})

In [17]:
results = []
for i in range(len(testing_dataset)):
    code = testing_dataset[i]
    label = code['label']
    input_ids = torch.tensor(code["input_ids"]).to(device)
    attention_mask = torch.tensor(code["attention_mask"]).to(device)
    with torch.no_grad():
        outputs = model(input_ids.unsqueeze(0), attention_mask.unsqueeze(0))
    y = np.argmax(outputs[0].to('cpu').numpy())
    results.append((label, y))

/var/folders/tf/97gp74_92k30dq5fwdw7j5bm0000gn/T/ipykernel_48409/1724829149.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_ids = torch.tensor(code["input_ids"]).to(device)
/var/folders/tf/97gp74_92k30dq5fwdw7j5bm0000gn/T/ipykernel_48409/1724829149.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  attention_mask = torch.tensor(code["attention_mask"]).to(device)


In [18]:
true_positive = 0
false_positive = 0
true_negative = 0
false_negative = 0

# Generates count of inference quadrants
for real, predicted in results:
    if real == 1 and predicted == 1:
        true_positive += 1
    elif real == 0 and predicted == 0:
        true_negative += 1
    elif real == 1 and predicted == 0:
        false_negative += 1
    elif real == 0 and predicted == 1:
        false_positive += 1

# print(l2)
print("True Positive:", true_positive)
print("True Negative:", true_negative)
print("False Positive:", false_positive)
print("False Negative:", false_negative)

# Machine Learning statistics and visuals. https://towardsdatascience.com/accuracy-recall-precision-f-score-specificity-which-to-optimize-on-867d3f11124
print("Accuracy:", (true_positive + true_negative)/(true_positive + true_negative + false_positive + false_negative))
precision = true_positive/(true_positive + false_positive)
print("Precision:", precision)
recall = true_positive/(true_positive + false_negative)
print("Recall:", recall)
print("F1-score", 2*(recall * precision)/(recall + precision))
print("Specificity:", true_negative/(true_negative + false_positive))

True Positive: 13
True Negative: 497
False Positive: 4
False Negative: 2
Accuracy: 0.9883720930232558
Precision: 0.7647058823529411
Recall: 0.8666666666666667
F1-score 0.8125
Specificity: 0.9920159680638723


[(tensor(0), 0),
 (tensor(1), 1),
 (tensor(0), 0),
 (tensor(1), 1),
 (tensor(1), 1),
 (tensor(1), 1),
 (tensor(1), 1),
 (tensor(1), 1),
 (tensor(0), 0),
 (tensor(1), 1),
 (tensor(1), 1),
 (tensor(1), 1),
 (tensor(0), 0),
 (tensor(1), 1),
 (tensor(1), 1),
 (tensor(1), 1),
 (tensor(1), 1),
 (tensor(1), 1),
 (tensor(0), 0),
 (tensor(0), 0),
 (tensor(1), 1)]

In [ ]:
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sn
real = []
predicted = []
for r, p in results:
    real.append(r)
    predicted.append(p)
cf_matrix = confusion_matrix(real, predicted)
df_cm = pd.DataFrame(cf_matrix, index=[0, 1],
                     columns=[0, 1])
plt.figure(figsize=(12, 7))
sn.heatmap(df_cm, annot=True)

In [20]:
torch.save(model.state_dict(), '../data/opcodes_model_weights.pth')

In [ ]:
model.load_state_dict(torch.load("/home/venglov/Documents/model_weights.pth"))

In [ ]:
address = "0x792e8f3727cad6e00c58d478798f0907c4cec340"


In [ ]:
model.eval()


In [ ]:
model.to(device)

In [ ]:
tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")


In [ ]:
web3 = Web3(Web3.HTTPProvider("https://eth-mainnet.gateway.pokt.network/v1/lb/6266d6cdaa777e00391e0d29"))
code = web3.eth.getCode(Web3.toChecksumAddress(address))

In [ ]:
code = "0x6000602061034003526102006112e0602039600051803560f81c906001016000526002026020015160f01c565b5b00600051803560f81c906001016000526002026020015160f01c565b01600051803560f81c906001016000526002026020015160f01c565b02600051803560f81c906001016000526002026020015160f01c565b03600051803560f81c906001016000526002026020015160f01c565b04600051803560f81c906001016000526002026020015160f01c565b05600051803560f81c906001016000526002026020015160f01c565b06600051803560f81c906001016000526002026020015160f01c565b07600051803560f81c906001016000526002026020015160f01c565b08600051803560f81c906001016000526002026020015160f01c565b09600051803560f81c906001016000526002026020015160f01c565b0a600051803560f81c906001016000526002026020015160f01c565b0b600051803560f81c906001016000526002026020015160f01c565b10600051803560f81c906001016000526002026020015160f01c565b11600051803560f81c906001016000526002026020015160f01c565b12600051803560f81c906001016000526002026020015160f01c565b13600051803560f81c906001016000526002026020015160f01c565b14600051803560f81c906001016000526002026020015160f01c565b15600051803560f81c906001016000526002026020015160f01c565b16600051803560f81c906001016000526002026020015160f01c565b17600051803560f81c906001016000526002026020015160f01c565b18600051803560f81c906001016000526002026020015160f01c565b19600051803560f81c906001016000526002026020015160f01c565b1a600051803560f81c906001016000526002026020015160f01c565b1b600051803560f81c906001016000526002026020015160f01c565b1c600051803560f81c906001016000526002026020015160f01c565b1d600051803560f81c906001016000526002026020015160f01c565b6103400120600051803560f81c906001016000526002026020015160f01c565b30600051803560f81c906001016000526002026020015160f01c565b31600051803560f81c906001016000526002026020015160f01c565b32600051803560f81c906001016000526002026020015160f01c565b33600051803560f81c906001016000526002026020015160f01c565b34600051803560f81c906001016000526002026020015160f01c565b506000600051803560f81c906001016000526002026020015160f01c565b6000600051803560f81c906001016000526002026020015160f01c565b610340019050369037600051803560f81c906001016000526002026020015160f01c565b36600051803560f81c906001016000526002026020015160f01c565b6103400137600051803560f81c906001016000526002026020015160f01c565b3a600051803560f81c906001016000526002026020015160f01c565b3b600051803560f81c906001016000526002026020015160f01c565b9061034001903c600051803560f81c906001016000526002026020015160f01c565b3d600051803560f81c906001016000526002026020015160f01c565b610340013e600051803560f81c906001016000526002026020015160f01c565b3f600051803560f81c906001016000526002026020015160f01c565b40600051803560f81c906001016000526002026020015160f01c565b41600051803560f81c906001016000526002026020015160f01c565b42600051803560f81c906001016000526002026020015160f01c565b43600051803560f81c906001016000526002026020015160f01c565b44600051803560f81c906001016000526002026020015160f01c565b45600051803560f81c906001016000526002026020015160f01c565b46600051803560f81c906001016000526002026020015160f01c565b47600051803560f81c906001016000526002026020015160f01c565b48600051803560f81c906001016000526002026020015160f01c565b50600051803560f81c906001016000526002026020015160f01c565b6103400151600051803560f81c906001016000526002026020015160f01c565b6103400152600051803560f81c906001016000526002026020015160f01c565b6103400153600051803560f81c906001016000526002026020015160f01c565b54600051803560f81c906001016000526002026020015160f01c565b55600051803560f81c906001016000526002026020015160f01c565b803560f81c90600101600052605b14156106dd57600051803560f81c906001016000526002026020015160f01c565b60006000fd5b906106ae5750600051803560f81c906001016000526002026020015160f01c565b600160005103600051803560f81c906001016000526002026020015160f01c565b6000610340015903600051803560f81c906001016000526002026020015160f01c565b5a600051803560f81c906001016000526002026020015160f01c565b600051803560f81c906001016000526002026020015160f01c565b60016000518035826020036008021c9101600052600051803560f81c906001016000526002026020015160f01c565b60026000518035826020036008021c9101600052600051803560f81c906001016000526002026020015160f01c565b60036000518035826020036008021c9101600052600051803560f81c906001016000526002026020015160f01c565b60046000518035826020036008021c9101600052600051803560f81c906001016000526002026020015160f01c565b60056000518035826020036008021c9101600052600051803560f81c906001016000526002026020015160f01c565b60066000518035826020036008021c9101600052600051803560f81c906001016000526002026020015160f01c565b60076000518035826020036008021c9101600052600051803560f81c906001016000526002026020015160f01c565b60086000518035826020036008021c9101600052600051803560f81c906001016000526002026020015160f01c565b60096000518035826020036008021c9101600052600051803560f81c906001016000526002026020015160f01c565b600a6000518035826020036008021c9101600052600051803560f81c906001016000526002026020015160f01c565b600b6000518035826020036008021c9101600052600051803560f81c906001016000526002026020015160f01c565b600c6000518035826020036008021c9101600052600051803560f81c906001016000526002026020015160f01c565b600d6000518035826020036008021c9101600052600051803560f81c906001016000526002026020015160f01c565b600e6000518035826020036008021c9101600052600051803560f81c906001016000526002026020015160f01c565b600f6000518035826020036008021c9101600052600051803560f81c906001016000526002026020015160f01c565b60106000518035826020036008021c9101600052600051803560f81c906001016000526002026020015160f01c565b60116000518035826020036008021c9101600052600051803560f81c906001016000526002026020015160f01c565b60126000518035826020036008021c9101600052600051803560f81c906001016000526002026020015160f01c565b60136000518035826020036008021c9101600052600051803560f81c906001016000526002026020015160f01c565b60146000518035826020036008021c9101600052600051803560f81c906001016000526002026020015160f01c565b60156000518035826020036008021c9101600052600051803560f81c906001016000526002026020015160f01c565b60166000518035826020036008021c9101600052600051803560f81c906001016000526002026020015160f01c565b60176000518035826020036008021c9101600052600051803560f81c906001016000526002026020015160f01c565b60186000518035826020036008021c9101600052600051803560f81c906001016000526002026020015160f01c565b60196000518035826020036008021c9101600052600051803560f81c906001016000526002026020015160f01c565b601a6000518035826020036008021c9101600052600051803560f81c906001016000526002026020015160f01c565b601b6000518035826020036008021c9101600052600051803560f81c906001016000526002026020015160f01c565b601c6000518035826020036008021c9101600052600051803560f81c906001016000526002026020015160f01c565b601d6000518035826020036008021c9101600052600051803560f81c906001016000526002026020015160f01c565b601e6000518035826020036008021c9101600052600051803560f81c906001016000526002026020015160f01c565b601f6000518035826020036008021c9101600052600051803560f81c906001016000526002026020015160f01c565b60206000518035826020036008021c9101600052600051803560f81c906001016000526002026020015160f01c565b80600051803560f81c906001016000526002026020015160f01c565b81600051803560f81c906001016000526002026020015160f01c565b82600051803560f81c906001016000526002026020015160f01c565b83600051803560f81c906001016000526002026020015160f01c565b84600051803560f81c906001016000526002026020015160f01c565b85600051803560f81c906001016000526002026020015160f01c565b86600051803560f81c906001016000526002026020015160f01c565b87600051803560f81c906001016000526002026020015160f01c565b88600051803560f81c906001016000526002026020015160f01c565b89600051803560f81c906001016000526002026020015160f01c565b8a600051803560f81c906001016000526002026020015160f01c565b8b600051803560f81c906001016000526002026020015160f01c565b8c600051803560f81c906001016000526002026020015160f01c565b8d600051803560f81c906001016000526002026020015160f01c565b8e600051803560f81c906001016000526002026020015160f01c565b8f600051803560f81c906001016000526002026020015160f01c565b90600051803560f81c906001016000526002026020015160f01c565b91600051803560f81c906001016000526002026020015160f01c565b92600051803560f81c906001016000526002026020015160f01c565b93600051803560f81c906001016000526002026020015160f01c565b94600051803560f81c906001016000526002026020015160f01c565b95600051803560f81c906001016000526002026020015160f01c565b96600051803560f81c906001016000526002026020015160f01c565b97600051803560f81c906001016000526002026020015160f01c565b98600051803560f81c906001016000526002026020015160f01c565b99600051803560f81c906001016000526002026020015160f01c565b9a600051803560f81c906001016000526002026020015160f01c565b9b600051803560f81c906001016000526002026020015160f01c565b9c600051803560f81c906001016000526002026020015160f01c565b9d600051803560f81c906001016000526002026020015160f01c565b9e600051803560f81c906001016000526002026020015160f01c565b9f600051803560f81c906001016000526002026020015160f01c565b61034001a0600051803560f81c906001016000526002026020015160f01c565b61034001a1600051803560f81c906001016000526002026020015160f01c565b61034001a2600051803560f81c906001016000526002026020015160f01c565b61034001a3600051803560f81c906001016000526002026020015160f01c565b61034001a4600051803560f81c906001016000526002026020015160f01c565b906103400190f0600051803560f81c906001016000526002026020015160f01c565b926103400192946103400194f1600051803560f81c906001016000526002026020015160f01c565b926103400192946103400194f2600051803560f81c906001016000526002026020015160f01c565b61034001f35b6020803803610220396102205173ffffffffffffffffffffffffffffffffffffffff16301461124857916103400191936103400193f4600051803560f81c906001016000526002026020015160f01c565b60006000fd5b906103400190f5600051803560f81c906001016000526002026020015160f01c565b916103400191936103400193fa600051803560f81c906001016000526002026020015160f01c565b61034001fd600051803560f81c906001016000526002026020015160f01c565bfe600051803560f81c906001016000526002026020015160f01c565b60006000fd5b60006000fd002d004900650081009d00b900d500f1010d01290145016112da12da12da12da017d019901b501d101ed020902250241025d0279029502b102cd02e912da12da030512da12da12da12da12da12da12da12da12da12da12da12da12da12da12da03250341035d0379039503b103cf03ec0410042c044c0468048404a604c204e204fe051a05360552056e058a05a605c205de12da12da12da12da12da12da12da05fa0616063606560676069206ae06e3070407250748076412da12da12da12da077f07ae07dd080c083b086a089908c808f709260955098409b309e20a110a400a6f0a9e0acd0afc0b2b0b5a0b890bb80be70c160c450c740ca30cd20d010d300d5f0d7b0d970db30dcf0deb0e070e230e3f0e5b0e770e930eaf0ecb0ee70f030f1f0f3b0f570f730f8f0fab0fc70fe30fff101b10371053106f108b10a710c310df10ff111f113f115f12da12da12da12da12da12da12da12da12da12da12da12da12da12da12da12da12da12da12da12da12da12da12da12da12da12da12da12da12da12da12da12da12da12da12da12da12da12da12da12da12da12da12da12da12da12da12da12da12da12da12da12da12da12da12da12da12da12da12da12da12da12da12da12da12da12da12da12da12da12da12da12da12da12da12da117f11a111c911f111f7124e12da12da12da12da127012da12da129812b812d4000000000000000000000000cb70efa43300cd9b7ef4ed2087cea7f7f6f3c195"

In [ ]:
decompiled = preprocess_code(simple_web3_code_decompile(code))

In [ ]:

decompiled

In [ ]:
inputs = tokenizer([decompiled], padding="max_length", truncation=True)


In [ ]:
item = {key: torch.tensor(val[0]) for key, val in inputs.items()}

In [ ]:
input_ids = torch.tensor(item["input_ids"]).to(device)
attention_mask = torch.tensor(item["attention_mask"]).to(device)
with torch.no_grad():
    outputs = model(input_ids.unsqueeze(0), attention_mask.unsqueeze(0))
y = np.argmax(outputs[0].to('cpu').numpy())

In [ ]:
outputs[0]

In [ ]:
y